# Benchmarking [Mistral Medium 3](https://mistral.ai/news/mistral-medium-3)

[Here](https://docs.mistral.ai/getting-started/models/models_overview/) is the list of Mistral's models.

## Libraries

In [1]:
from mistralai import Mistral
from pathlib import Path
from time import sleep
import os
import pandas as pd
from tqdm.rich import tqdm
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score
)
import json
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.text import Text
import warnings

tqdm.pandas(desc="Fetching moderation scores")
console = Console()
warnings.filterwarnings("ignore")

## Global variables

In [ ]:
ROOT = Path('../..')
DATA_DIR = ROOT / "data"
BENCHMARK_PATH = DATA_DIR / "benchmark" / "benchmark_balanced_subset.csv"
output_path = DATA_DIR / "benchmark" / "mistral_medium_simple_prompt.csv"
API_KEY_PATH = DATA_DIR / "confidential" / "MISTRAL_API.txt"
console = Console()
model = "mistral-medium-2505"
system_prompt = (Path(".") / "API_SYSTEM_PROMPT_simple.txt").read_text().strip()
prompt = (Path(".") / "API_PROMPT_simple.txt").read_text().strip()

In [3]:
os.environ["HTTP_PROXY"] = "socks5h://127.0.0.1:1080"
os.environ["HTTPS_PROXY"] = "socks5h://127.0.0.1:1080"

In [4]:
client = Mistral(api_key=API_KEY_PATH.read_text().strip())

## Load dataset

In [5]:
df = pd.read_csv(BENCHMARK_PATH, encoding="utf-8")
df = df.dropna(subset=["content", "label"])
df["label"] = df["label"].astype(int)

label_counts = df["label"].value_counts().reset_index()
label_counts.columns = ["label", "count"]
table = Table(title="Label Counts", show_lines=True)
table.add_column("Label", justify="center", style="cyan")
table.add_column("Count", justify="center", style="yellow")
table.add_row("Total Rows", str(len(df)))
for _, row in label_counts.iterrows():
    table.add_row(str(row["label"]), str(row["count"]))
console.print(table, justify="left")
df.sample(5, random_state=42)

     Label Counts                                                                                                  
┏━━━━━━━━━━━━┳━━━━━━━┓                                                                                             
┃   Label    ┃ Count ┃                                                                                             
┡━━━━━━━━━━━━╇━━━━━━━┩                                                                                             
│ Total Rows │ 1388  │                                                                                             
├────────────┼───────┤                                                                                             
│     1      │  694  │                                                                                             
├────────────┼───────┤                                                                                             
│     0      │  694  │                                                                                             
└────────────┴───────┘

,msg_id,content,label
650,anon_msg_5ff0a5549f65,"des bovins mon pote, j'ai jamais vu ça sauf en...",0
208,anon_msg_2ea521264d58,Cet accent du sud dégueulasse,1
427,anon_msg_99e41015a911,"Joffrin, je serais capable de faire des trucs ...",1
677,anon_msg_31edc839b8b8,"D'accord le julf, mais vous n'êtes pas BLANC.",1
1272,anon_msg_15c39b3b7e9a,C'est Siemens Espagne hein... Pas le PDG du gr...,0


## Define prediction function

In [6]:
def create_batch_file(
        text: str,
        max_tokens: int = 20,
    ):
    input_text = prompt + f"« {text} »" + "\n Ce message est-il toxique ?\n"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": input_text}
        ],
        "max_tokens": max_tokens
        }

In [7]:
def create_json_batch(df, batch_path):

    json_batch = {}

    for i, row in df.iterrows():
        text = row["content"]
        if text in json_batch:
            continue

        json_batch[text] = {
            "custom_id": f"id_{i}",
            "body": create_batch_file(
                text=text
            )
        }

    batch_path.parent.mkdir(parents=True, exist_ok=True)
    with batch_path.open("w", encoding="utf-8") as f:
        for entry in json_batch.values():
            f.write(json.dumps(entry) + "\n")
    console.print(f"Batch file written to {batch_path}")

In [8]:
def upload_batch(batch_path) -> str:
    file = open(batch_path, "rb")

    uploaded = client.files.upload(
        file={
            "file_name": "batch_path",
            "content": file
        },
        purpose = "batch"
    )

    uploaded_file_id = uploaded.id

    console.print(f"[green]✔ Uploaded batch file. ID: [bold]{uploaded_file_id}[/bold]")

    return uploaded_file_id

In [9]:
def submit_batch(uploaded_file_id) -> str:

    batch_job = client.batch.jobs.create(
        input_files=[uploaded_file_id],
        model=model,
        endpoint="/v1/chat/completions",
    )

    console.print(f"[green]✔ Batch submitted. ID: [bold]{batch_job.id}[/bold]")
    return batch_job

In [10]:
def wait_for_completion(batch_job):
    elapsed = 0
    while batch_job.status in ["QUEUED", "RUNNING"]:
        batch_job = client.batch.jobs.get(job_id=batch_job.id)
        console.print(f"[yellow] Batch status (after {elapsed}s): [bold]{batch_job.status}[/bold]")
        sleep(60)
        elapsed += 60
    print(f"Batch job {batch_job.id} completed with status: {batch_job.status}")
    return batch_job

In [11]:
def download_and_parse_results(batch, output_path) -> pd.DataFrame:
    if batch.output_file is not None:
        print(f"Downloading file to {output_path}")
        output_file = client.files.download(file_id=batch.output_file)
        with open(output_path, "w") as f:
            for chunk in output_file.stream:
                f.write(chunk.decode("utf-8"))
        print(f"Downloaded file to {output_path}")
    
    content = output_path.read_bytes()

    results = []
    for line in content.decode("utf-8").splitlines():
        data = json.loads(line)
        if data.get("error"):
            console.print(f"[red] Error for {data['custom_id']}: {data['error']}")
            continue
        id = data["custom_id"]
        choice = data["response"]["body"]["choices"][0]["message"]["content"].strip()
        usage = data["response"]["body"].get("usage", {})
        results.append({
            "id": id,
            "conversation": choice,
        })
    return pd.DataFrame(results)

In [12]:
def pipeline_for(df):
    create_json_batch(
        df,
        Path(f"batch_{model}.jsonl")
    )
    uploaded_file_id = upload_batch(Path(f"batch_{model}.jsonl"))
    batch = submit_batch(uploaded_file_id)
    batch = wait_for_completion(batch)
    df_answers = download_and_parse_results(batch, Path(f"answers_{model}.jsonl"))
    return df_answers

In [13]:
results = pipeline_for(df)

Batch file written to batch_mistral-medium-2505.jsonl

✔ Uploaded batch file. ID: 071c631d-2bfd-4e63-a22d-a011fe7894ad

✔ Batch submitted. ID: dea0fff5-94b6-43b7-971a-20d4aa02bd6a

 Batch status (after 0s): RUNNING

 Batch status (after 60s): RUNNING

...

 Batch status (after 180s): RUNNING

 Batch status (after 240s): RUNNING

 Batch status (after 300s): RUNNING

 Batch status (after 360s): SUCCESS

Batch job dea0fff5-94b6-43b7-971a-20d4aa02bd6a completed with status: SUCCESS
Downloaded file to answers_mistral-medium-2505.jsonl


In [14]:
df['id'] = [f"id_{i}" for i in range(len(df))]
results.sort_values(by='id', inplace=True)
df.sort_values(by='id', inplace=True)
df['toxicity_score'] = results['conversation'].tolist()

In [15]:
df['toxicity_score'] = results['conversation'].tolist()

In [16]:
df

,msg_id,content,label,id,toxicity_score
0,anon_msg_468ffd36870c,AYAAAAAAAAAAAA LES CRACKHEAD ANTI MIGRANT BORD...,1,id_0,toxique
1,anon_msg_c30ce8c67bac,Bordel le gars chantait 'FN souffrance' alors ...,1,id_1,toxique
10,anon_msg_4446a6421aed,Tous les débiles qui publient du contenu pour ...,1,id_10,toxique
100,anon_msg_41a115e1fdb3,pays ultrapauvre à l'islam archi-rigoriste\nét...,1,id_100,toxique
1000,anon_msg_f75718ce6344,Ils ont raison\nLes gauchistes en sueur devant...,0,id_1000,toxique
...,...,...,...,...,...
995,anon_msg_051f30d6f917,"Bah oui, il est où le problème ?\nSi j'ai plus...",1,id_995,toxique
996,anon_msg_6b90c4e1b70c,Je suis étonné que personne ne fasse un dessin...,0,id_996,non-toxique
997,anon_msg_f781e76d1745,"Les nazis c'est fini depuis 45, faut arrêter a...",1,id_997,non-toxique
998,anon_msg_58e47fd26d87,Par contre le Jap sur 20m il les tabasse\n:noel:,0,id_998,toxique


In [17]:
df['prediction'] = df['toxicity_score'].apply(lambda x: 1 if x.lower() == "toxique" else 0)

In [18]:
for i, row in df.sample(5, random_state=42).iterrows():
    content = Text(row['content'], style="bold")
    toxicity = f"[yellow]Toxicity Score:[/yellow] [bold]{row['prediction']}[/bold]"
    label = f"[cyan]Label:[/cyan] [bold]{row['label']}[/bold]"
    panel = Panel.fit(
        f"{content}\n\n{toxicity}\n{label}",
        title=f"Exemple {i+1}",
        border_style="magenta"
    )
    console.print(panel)

╭────────────────────────────────────────────────── Exemple 335 ──────────────────────────────────────────────────╮
│ Contre son camp, on n'oublie pas que les Grecs pratiquaient la pédérastie, et bâtissaient des statues d'hommes  │
│ dénudés.                                                                                                        │
│                                                                                                                 │
│ Toxicity Score: 1                                                                                               │
│ Label: 1                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────── Exemple 1186 ────────╮
│ Ca marche pas si tu fais ça │
│                             │
│ Toxicity Score: 0           │
│ Label: 0                    │
╰─────────────────────────────╯

╭──────────── Exemple 1383 ─────────────╮
│ Le retour à la monarchie change tout. │
│ Sortir de l'UE change tout.           │
│                                       │
│ Toxicity Score: 0                     │
│ Label: 0                              │
╰───────────────────────────────────────╯

╭───────────────────────────── Exemple 360 ─────────────────────────────╮
│ Voilà pourquoi il faut se débarrasser de vous en France et en Europe. │
│ On veut pas d'arriérés de votre genre.                                │
│                                                                       │
│ Toxicity Score: 1                                                     │
│ Label: 1                                                              │
╰───────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Exemple 896 ─────────────────────────────────────────────╮
│ Sincèrement je pense qu'il a un gros problème psycatrique le mec qui boucle depuis 2 ans sur le topic │
│                                                                                                       │
│ Toxicity Score: 1                                                                                     │
│ Label: 0                                                                                              │
╰───────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Metrics & Report        

| Metric                     | Formula                                           | Interpretation                                                                                                       |
| -------------------------- | ------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **Precision**              | `TP / (TP + FP)`                                  | Of the samples predicted **toxic**, how many were **actually toxic**? <br>→ High precision = **low false positives** |
| **Recall** *(Sensitivity)* | `TP / (TP + FN)`                                  | Of the **actual toxic** samples, how many did we **correctly identify**? <br>→ High recall = **low false negatives** |
| **F1-score**               | `2 * (Precision * Recall) / (Precision + Recall)` | Harmonic mean of precision and recall. <br>→ Best when **balance** is needed                                         |
| **Accuracy**               | `(TP + TN) / (TP + TN + FP + FN)`                 | Fraction of all correct predictions (toxic and non-toxic). <br>→ Can be misleading on imbalanced data                |
| **ROC AUC**                | Area under the ROC Curve                          | Measures the **ranking ability** of the classifier. <br>→ Higher = better separation of toxic vs. non-toxic          |


In [19]:
y_true = df["label"]
y_pred = df["prediction"]

In [20]:
# Rapport de classification
report = classification_report(y_true, y_pred, digits=3, output_dict=True)
table = Table(title="Classification Report", show_lines=True)
table.add_column("Classe", style="cyan", justify="center")
table.add_column("Precision", justify="center")
table.add_column("Recall", justify="center")
table.add_column("F1-score", justify="center")
table.add_column("Support", justify="center")

for label, metrics in report.items():
    if label in ["accuracy", "macro avg", "weighted avg"]:
        continue
    table.add_row(
        str(label),
        f"{metrics['precision']:.3f}",
        f"{metrics['recall']:.3f}",
        f"{metrics['f1-score']:.3f}",
        f"{int(metrics['support'])}"
    )
# Ajout de l'accuracy
table.add_row(
    "[bold yellow]Accuracy[/bold yellow]",
    "-",
    "-",
    f"{report['accuracy']:.3f}",
    "-"
)
console.print(table)

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
cm_table = Table(title="Matrice de Confusion", show_lines=True)
cm_table.add_column(" ", style="bold")
cm_table.add_column("Prédit 0", justify="center")
cm_table.add_column("Prédit 1", justify="center")
cm_table.add_row("Réel 0", str(cm[0, 0]), str(cm[0, 1]))
cm_table.add_row("Réel 1", str(cm[1, 0]), str(cm[1, 1]))
console.print(cm_table)

# ROC AUC
try:
    auc = roc_auc_score(y_true, df["prediction"])
    console.print(Panel(f"[bold green]ROC AUC Score : {auc:.3f}[/bold green]", title="ROC AUC"))
except:
    console.print(Panel("[bold red]Impossible de calculer le ROC AUC.[/bold red]", title="ROC AUC"))

                Classification Report                 
┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃  Classe  ┃ Precision ┃ Recall ┃ F1-score ┃ Support ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│    0     │   0.971   │ 0.679  │  0.799   │   694   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│    1     │   0.753   │ 0.980  │  0.852   │   694   │
├──────────┼───────────┼────────┼──────────┼─────────┤
│ Accuracy │     -     │   -    │  0.829   │    -    │
└──────────┴───────────┴────────┴──────────┴─────────┘

      Matrice de Confusion      
┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃        ┃ Prédit 0 ┃ Prédit 1 ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Réel 0 │   471    │   223    │
├────────┼──────────┼──────────┤
│ Réel 1 │    14    │   680    │
└────────┴──────────┴──────────┘

╭──────────────────────────────────────────────────── ROC AUC ────────────────────────────────────────────────────╮
│ ROC AUC Score : 0.829                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Save the predictions

In [21]:
df

,msg_id,content,label,id,toxicity_score,prediction
0,anon_msg_468ffd36870c,AYAAAAAAAAAAAA LES CRACKHEAD ANTI MIGRANT BORD...,1,id_0,toxique,1
1,anon_msg_c30ce8c67bac,Bordel le gars chantait 'FN souffrance' alors ...,1,id_1,toxique,1
10,anon_msg_4446a6421aed,Tous les débiles qui publient du contenu pour ...,1,id_10,toxique,1
100,anon_msg_41a115e1fdb3,pays ultrapauvre à l'islam archi-rigoriste\nét...,1,id_100,toxique,1
1000,anon_msg_f75718ce6344,Ils ont raison\nLes gauchistes en sueur devant...,0,id_1000,toxique,1
...,...,...,...,...,...,...
995,anon_msg_051f30d6f917,"Bah oui, il est où le problème ?\nSi j'ai plus...",1,id_995,toxique,1
996,anon_msg_6b90c4e1b70c,Je suis étonné que personne ne fasse un dessin...,0,id_996,non-toxique,0
997,anon_msg_f781e76d1745,"Les nazis c'est fini depuis 45, faut arrêter a...",1,id_997,non-toxique,0
998,anon_msg_58e47fd26d87,Par contre le Jap sur 20m il les tabasse\n:noel:,0,id_998,toxique,1


In [22]:
output_path

PosixPath('../data/benchmark/mistral_medium_simple_prompt.csv')

In [23]:
df.to_csv(output_path, index=False, encoding="utf-8")